# Summary Data Visualisation Creation
The purpose of this Jupyter Notebook is to create an interactive plot which summarises all the stock price data and reddit data from r/WallStreetBets using Bokeh.

### Processing Reddit Data

In [143]:
import os
import json

import pandas as pd
from bokeh.models import Range1d
from bokeh.transform import transform
from bokeh.models.tools import HoverTool
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter, Title, LinearAxis

Loading pickled GME stock price dataframe

In [144]:
pickle_file_path = '../scraping/gme_daily_transformed_df.pkl'
gme_daily_transformed_df = pd.read_pickle(pickle_file_path)

# Filtering the DataFrame to include only the data from January 2021 to April 2021
gme_jan_apr2021_df = gme_daily_transformed_df[(gme_daily_transformed_df['Date'] >= '2021-01-01') & (gme_daily_transformed_df['Date'] <= '2021-04-01')].copy()

Reading Reddit data from CSV files into a dataframe

In [145]:
data1_df = pd.read_csv('../data/reddit_data/Submissions_2021-01_FilteredBySubreddit_GME.csv')
data2_df = pd.read_csv('../data/reddit_data/Submissions_2021-02_FilteredBySubreddit_GME.csv')
data3_df = pd.read_csv('../data/reddit_data/Submissions_2021-03_FilteredBySubreddit_GME.csv')

df_all_reddit_data = pd.concat([data1_df, data2_df, data3_df], axis = 0,
                        ignore_index=True)


Before processing the dataframe

In [146]:
df_all_reddit_data.head()

,submission_id,redditor_name,created_at,title,text,subreddit,permalink,attachment,flair,score,num_comments,edited
0,ko124i,[deleted],2021-01-01T00:02:06,"3k - 170k since March (Also, buy LIT!!)",[deleted],wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,NaN,"{'link': 'Gain', 'author': None}",{'2021-06-10T13:50:58': 34},{'2021-06-10T13:50:58': 14},False
1,ko12uq,[deleted],2021-01-01T00:03:20,Got out of PLTR calls after learning about IV ...,[deleted],wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,{'url': 'https://www.reddit.com/gallery/ko12uq'},"{'link': 'Gain', 'author': None}",{'2021-06-10T13:50:59': 2},{'2021-06-10T13:50:59': 0},False
2,ko13df,[deleted],2021-01-01T00:04:11,Hell of a headline,[deleted],wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,{'png': 'https://i.redd.it/620igsuk3m861.png'},"{'link': 'Meme', 'author': None}",{'2021-06-10T13:50:59': 14},{'2021-06-10T13:50:59': 7},False
3,ko13q2,DarkCookie243,2021-01-01T00:04:47,A message from JPow for New Years Eve to all o...,NaN,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,{'jpg': 'https://i.redd.it/rkb331xu3m861.jpg'},"{'link': 'Meme', 'author': None}",{'2021-06-10T13:51:00': 4},{'2021-06-10T13:51:00': 0},False
4,ko145e,stevenconrad,2021-01-01T00:05:29,"GME to 420.69, but only if we make it happen. ...",[removed],wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,NaN,"{'link': 'DD', 'author': None}",{'2021-06-10T13:51:00': 9},{'2021-06-10T13:51:00': 4},False


Processing dataframe

In [147]:
# Extracting the numerical part after the colon and converting to integer
df_all_reddit_data['num_comments'] = df_all_reddit_data['num_comments'].str.extract(r': (\d+)}').astype(int)
df_all_reddit_data['score'] = df_all_reddit_data['score'].str.extract(r': (\d+)}').astype(int)

# Converting and normalising the created_at column
df_all_reddit_data['created_at'] = pd.to_datetime(df_all_reddit_data['created_at'])
df_all_reddit_data['created_at'] = df_all_reddit_data['created_at'].dt.normalize()


After processing the dataframe

In [148]:
df_all_reddit_data.head()


,submission_id,redditor_name,created_at,title,text,subreddit,permalink,attachment,flair,score,num_comments,edited
0,ko124i,[deleted],2021-01-01,"3k - 170k since March (Also, buy LIT!!)",[deleted],wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,NaN,"{'link': 'Gain', 'author': None}",34,14,False
1,ko12uq,[deleted],2021-01-01,Got out of PLTR calls after learning about IV ...,[deleted],wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,{'url': 'https://www.reddit.com/gallery/ko12uq'},"{'link': 'Gain', 'author': None}",2,0,False
2,ko13df,[deleted],2021-01-01,Hell of a headline,[deleted],wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,{'png': 'https://i.redd.it/620igsuk3m861.png'},"{'link': 'Meme', 'author': None}",14,7,False
3,ko13q2,DarkCookie243,2021-01-01,A message from JPow for New Years Eve to all o...,NaN,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,{'jpg': 'https://i.redd.it/rkb331xu3m861.jpg'},"{'link': 'Meme', 'author': None}",4,0,False
4,ko145e,stevenconrad,2021-01-01,"GME to 420.69, but only if we make it happen. ...",[removed],wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,NaN,"{'link': 'DD', 'author': None}",9,4,False


Groupby - keep looking into

In [149]:
df_all_reddit_data.rename(columns={'created_at': 'Date'}, inplace=True)
df_grouped = df_all_reddit_data.groupby('Date')

df_grouped = df_all_reddit_data.groupby('Date').size().reset_index(name='post_count')

df_comments_count = df_all_reddit_data.groupby('Date')['num_comments'].sum().reset_index(name='total_comments')

# Merge 'df_grouped' with 'df_comments_count' on 'Date'
df_grouped = df_grouped.merge(df_comments_count, on='Date', how='left')

df_grouped

,Date,post_count,total_comments
0,2021-01-01,589,45186
1,2021-01-02,625,11609
2,2021-01-03,613,25134
3,2021-01-04,1029,55169
4,2021-01-05,965,65804
...,...,...,...
86,2021-03-28,1178,30131
87,2021-03-29,2065,85797
88,2021-03-30,2105,91976
89,2021-03-31,2147,82231


In [150]:
merged_df = df_grouped.merge(gme_jan_apr2021_df, on='Date', how='outer')
merged_df

,Date,post_count,total_comments,Open,High,Low,Close,Volume
0,2021-01-01,589,45186,NaN,NaN,NaN,NaN,NaN
1,2021-01-02,625,11609,NaN,NaN,NaN,NaN,NaN
2,2021-01-03,613,25134,NaN,NaN,NaN,NaN,NaN
3,2021-01-04,1029,55169,19.00,19.1000,17.1500,17.25,10022474.0
4,2021-01-05,965,65804,17.35,18.0765,17.2300,17.37,4961457.0
...,...,...,...,...,...,...,...,...
86,2021-03-28,1178,30131,NaN,NaN,NaN,NaN,NaN
87,2021-03-29,2065,85797,180.75,193.9200,173.5100,181.30,10042175.0
88,2021-03-30,2105,91976,187.50,204.3000,182.0000,194.46,17094924.0
89,2021-03-31,2147,82231,197.50,199.4600,187.1102,189.82,8393834.0


In [151]:
cleaned_df = merged_df.dropna().copy()
cleaned_df['Volume'] = cleaned_df['Volume'].astype(int)
# Remove the last row using .iloc
cleaned_df = cleaned_df.iloc[:-1]
cleaned_df

,Date,post_count,total_comments,Open,High,Low,Close,Volume
3,2021-01-04,1029,55169,19.00,19.1000,17.1500,17.25,10022474
4,2021-01-05,965,65804,17.35,18.0765,17.2300,17.37,4961457
5,2021-01-06,1225,77419,17.34,18.9800,17.3300,18.36,6056248
6,2021-01-07,1399,68127,18.47,19.4500,18.0200,18.08,6129276
7,2021-01-08,1551,95624,18.18,18.3000,17.0800,17.69,6481960
...,...,...,...,...,...,...,...,...
83,2021-03-25,3493,146707,123.49,187.5000,116.9000,183.75,49926442
84,2021-03-26,2986,148208,197.68,218.9344,163.2600,181.00,37430672
87,2021-03-29,2065,85797,180.75,193.9200,173.5100,181.30,10042175
88,2021-03-30,2105,91976,187.50,204.3000,182.0000,194.46,17094924


### Plotting the visualisation

In [152]:
cleaned_df['Date'] = pd.to_datetime(cleaned_df['Date'])
cleaned_df.sort_values('Date', inplace=True)

source = ColumnDataSource(cleaned_df)

color_mapper = LinearColorMapper(palette="Viridis256", low=cleaned_df['total_comments'].min(), high=cleaned_df['total_comments'].max())

scale_factor = 0.0015
max_size = 10000
min_size = 5
cleaned_df['size'] = cleaned_df['post_count'] * scale_factor
cleaned_df['size'] = cleaned_df['size'].clip(lower=min_size, upper=max_size)

source.data['size'] = cleaned_df['size']

p = figure(width=1450, height=700, x_axis_type="datetime")

title = Title(text="GameStop Trading Frenzy: A Synchronized Surge in Stock Price, Volume, and Reddit Activity", text_font_size="16pt", align="center")

p.circle(x='Date', y='Volume', size='size', source=source, color=transform('total_comments', color_mapper), alpha=0.7)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title='Total Comments', formatter=NumeralTickFormatter(format="0,0"))
p.add_layout(color_bar, 'below')

p.yaxis.formatter = NumeralTickFormatter(format="0a")
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Trading Volume (in millions)'

p.extra_y_ranges = {'Close Price': Range1d(start=cleaned_df['Close'].min(), end=400)}

p.line('Date', 'Close', source=source, color='red', y_range_name='Close Price', legend_label='Close Price')
p.add_layout(LinearAxis(y_range_name='Close Price', axis_label='Close Price'), 'right')

hover = HoverTool(tooltips=[("Date", "@Date{%F}"),
                            ("Volume", "@Volume"),
                            ("Post Count", "@post_count"),
                            ("Total Comments", "@total_comments"),
                            ("Close Price", "@Close")],
                  formatters={'@Date': 'datetime'})
p.add_tools(hover)

p.title = title

output_file("interactive_plot.html")
output_file("../docs/interactive_plot.html")
save(p)


'c:\\Users\\annaj\\Desktop\\DS105\\ds105a-project-data-dabblers\\docs\\interactive_plot.html'